In [ ]:
import pandas as pd;
import tensorflow as tf;
import cv2;
import numpy as np;
import matplotlib.pyplot as plt;
import os;
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K



In [ ]:
def changetoBinary(img):
        image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        _, binary = cv2.threshold(gray,110, 255, cv2.THRESH_BINARY_INV)
        binary = cv2.cvtColor(binary, cv2.COLOR_GRAY2RGB)
#         plt.imshow(binary)
#         plt.show()
        binary  = cv2.resize(binary, (75, 75)) 

        
#         print(binary.shape)
#         print(gray.shape)
        
        return binary

In [ ]:
train_image_dir ="/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/";
image_loc=train_image_dir+"A/"+os.listdir(train_image_dir+"A/")[0]

image=cv2.imread(image_loc)
# plt.imshow(image);
# plt.show()
print(image.shape)
IMG_HEIGHT=75;
IMG_WIDTH=75
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
_, binary = cv2.threshold(gray,125, 255, cv2.THRESH_BINARY_INV)
plt.imshow(binary);


In [ ]:

train_image_generator = ImageDataGenerator(rescale=1.0/255,
                                           rotation_range=20,
                                           width_shift_range=.15,
                                           height_shift_range=.15,
                                           zoom_range=0.5,
                                           validation_split=0.2,
                                           preprocessing_function=changetoBinary)
train_image_generator.preprocessing_function = changetoBinary

train_data_gen = train_image_generator.flow_from_directory(batch_size=100,
                                                           directory=train_image_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode="categorical",
                                                           subset='training')

In [ ]:


validation_data_gen = train_image_generator.flow_from_directory(batch_size=100,
                                                           directory=train_image_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode="categorical",
                                                           subset='validation')

In [ ]:
alphabats= ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','del','space','nothing']
print(len(train_data_gen))
plt.imshow(255.0*train_data_gen[0][0][0])
plt.show()
print(train_data_gen[0])

In [ ]:
checkpoint = ModelCheckpoint("/kaggle/working/best_model.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=1)


In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(16, 3, padding='same', activation='relu', 
           input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(29,activation='softmax')
])

In [ ]:

model.compile(optimizer='adam',
             loss="categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=696,
    epochs=10,
    validation_data=validation_data_gen,
    validation_steps=174,
    callbacks=[checkpoint]
)

In [ ]:
model.save("/kaggle/working/")
# history.save("/kaggle/working/")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']
# no_of_epoch = 10
epochs_range = range(10) 


plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save("/kaggle/working/my_model.h5")

In [ ]:
new_model = tf.keras.models.load_model('/kaggle/working/my_model.h5')
new_model.summary()

In [ ]:
# imgdir= "/home/praveen/ml/sign_language/asl_datbase/asl_alphabet_test/asl_alphabet_test/A_test.jpg"
imgdir="/kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/J_test.jpg";


In [ ]:
img= cv2.imread(imgdir)
plt.imshow(img)
plt.show()

img= changetoBinary(img)
# print(img)
plt.imshow(img)
plt.show()



In [ ]:
img=tf.reshape(img,[1,75, 75,3])

print(img.shape)


In [ ]:
images=[]
images.append(img)
pred = new_model.predict(img)
pred1=new_model.predict_classes(images)
alphabats[pred1[0]]